In [5]:
# !pip install dask

In [9]:
import pandas as pd
import numpy as np

In [10]:
pdf = pd.read_csv("/kaggle/input/paydown/sample_data_for_PayDown.csv")

In [11]:
behv_usage_pdf = pdf.loc[:, ["uniq_key", "mnth_cd", "behv", "Total Usage", "profit"]].rename(columns={"Total Usage":"usage"})

In [12]:
behv_usage_pdf["uniq_key"].nunique()

114

In [13]:
cust_202312 = behv_usage_pdf.query("mnth_cd == 202312")["uniq_key"].drop_duplicates()
cust_202301 = behv_usage_pdf.query("mnth_cd == 202301")["uniq_key"].drop_duplicates()

In [14]:
cust_202312.count()

114

In [15]:
cust_202301.count()

104

In [20]:
cust_12_mth = cust_202312[cust_202312.isin(cust_202301)].to_frame()

In [22]:
card_behv_12_mnth = behv_usage_pdf.merge(cust_12_mth, on=["uniq_key"])

In [23]:
card_behv_12_mnth["uniq_key"].nunique()

104

In [24]:
pay_down_flg = (card_behv_12_mnth
 .assign(pay_down_flg = lambda x :  (x["behv"] == "Revolver") & (x["usage"] == 0) )
)

In [25]:
pay_down_flag_list = (pay_down_flg
 .sort_values(["uniq_key", "mnth_cd"], ascending=False)
 .groupby("uniq_key", as_index=False)
 .agg(pay_down_flg_list = ("pay_down_flg", list),
      profit = ("profit", "sum")
     )
 .astype({"pay_down_flg_list":"string"})
)

In [26]:
with pd.option_context('display.max_colwidth', None):
    display(pay_down_flag_list[pay_down_flag_list["uniq_key"] == 4855])

,uniq_key,pay_down_flg_list,profit
102,4855,"[True, True, True, True, False, False, False, False, False, False, False, False]",14183.552944


In [27]:
pay_down_flag_list[pay_down_flag_list["pay_down_flg_list"] == "[True, True, True, True, False, False, False, False, False, False, False, False]"]

,uniq_key,pay_down_flg_list,profit
4,187,"[True, True, True, True, False, False, False, ...",293.887386
102,4855,"[True, True, True, True, False, False, False, ...",14183.552944


In [28]:
profit_group = (pay_down_flag_list
 .groupby("pay_down_flg_list")
 .agg(n_card = ("uniq_key", "count"),
      tt_profit = ("profit", "sum")
     )
)

In [29]:
profit_group.sort_index(ascending=False)

,n_card,tt_profit
pay_down_flg_list,,
"[True, True, True, True, True, True, True, True, True, True, True, True]",1,6015.723894
"[True, True, True, True, True, True, True, True, True, True, True, False]",1,1396.106840
"[True, True, True, True, True, True, True, True, True, False, False, False]",1,-27.582475
"[True, True, True, True, True, True, True, False, True, False, False, False]",1,1139.704091
"[True, True, True, True, True, True, True, False, False, False, False, False]",1,17346.056158
...,...,...
"[False, False, False, False, False, False, False, False, False, True, False, True]",1,7405.737179
"[False, False, False, False, False, False, False, False, False, True, False, False]",1,9082.934124
"[False, False, False, False, False, False, False, False, False, False, True, False]",3,5845.243784


## Create Lead column (shift(-x))

In [30]:
pay_down_lead = (pay_down_flg
 .sort_values(["uniq_key", "mnth_cd"], ascending=False)
 .assign(curr = lambda x : x["pay_down_flg"])
 .assign(lead1 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-1))
 .assign(lead2 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-2))
 .assign(lead3 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-3))
 .assign(lead4 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-4))
 .assign(lead5 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-5))
 .assign(lead6 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-6))
 .assign(lead7 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-7))
 .assign(lead8 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-8))
 .assign(lead9 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-9))
 .assign(lead10 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-10))
 .assign(lead11 = lambda x : x.groupby("uniq_key")["pay_down_flg"].shift(-11))
 .query("mnth_cd == 202312")
)

In [31]:
pay_down_lead.head()

,uniq_key,mnth_cd,behv,usage,profit,pay_down_flg,curr,lead1,lead2,lead3,lead4,lead5,lead6,lead7,lead8,lead9,lead10,lead11
1146,4861,202312,Revolver,0.0,186.930860,True,True,True,True,True,True,True,False,True,True,True,True,True
1162,4855,202312,Revolver,0.0,3122.936243,True,True,True,True,True,False,False,False,False,False,False,False,False
1151,4848,202312,Revolver,0.0,-88.860153,True,True,True,True,False,False,False,False,False,False,False,False,True
1230,4697,202312,Inactive,503.0,-132.636638,False,False,False,False,False,False,False,False,False,False,False,False,False
1183,4526,202312,Inactive,0.0,0.000000,False,False,False,False,False,False,False,False,True,True,True,True,False


In [32]:
card_behv_12_mnth[card_behv_12_mnth["uniq_key"]==4855].sort_values("mnth_cd", ascending=False)

,uniq_key,mnth_cd,behv,usage,profit
1162,4855,202312,Revolver,0.00,3122.936243
1058,4855,202311,Revolver,0.00,3116.264802
954,4855,202310,Revolver,0.00,2417.558213
850,4855,202309,Revolver,0.00,3270.233883
746,4855,202308,Revolver,66881.77,2039.341660
642,4855,202307,Revolver,554386.41,5327.613489
538,4855,202306,Revolver,95381.62,3390.882576
434,4855,202305,Revolver,85671.92,428.448091
330,4855,202304,Transactor,47300.24,-12356.100080
226,4855,202303,Transactor,33762.20,516.618416
